In [ ]:
import json
import time
import requests
import telepot
from telepot.loop import MessageLoop
import paho.mqtt.client as mqtt
import os

In [ ]:
class NotificationBot:
    def __init__(self, settings):
        self.settings = settings
        self.token = settings["telegramToken"]
        
        # MQTT Setup
        self.mqtt_broker = os.getenv('MQTT_BROKER_HOST', 'mosquitto')
        self.mqtt_port = int(os.getenv('MQTT_BROKER_PORT', 1883))
        self.topic = "building/observation"

        # Buffer to store the latest data received from MQTT
        self.last_observation = {}

        # Initialize MQTT Client
        self.mqtt_client = mqtt.Client()
        self.mqtt_client.on_connect = self.on_mqtt_connect
        self.mqtt_client.on_message = self.on_mqtt_message
        self.mqtt_client.connect(self.mqtt_broker, self.mqtt_port)
        self.mqtt_client.loop_start()

        # Config mapping for zones
        self.zones = ["Tair_z1", "Tair_z2", "Tair_z4"]
        self.shades = {
            "Zone 1": ["ShadeStatus_Zone1_Wall2", "ShadeStatus_Zone1_Wall8", "ShadeStatus_Zone1_Wall9"],
            "Zone 2": ["ShadeStatus_Zone2_Wall2", "ShadeStatus_Zone2_wall3"],
            "Zone 4": ["ShadeStatus_Zone4_Wall2"]
        }

        # Comfort thresholds
        self.t_min = 19
        self.t_max = 24

        self.bot = telepot.Bot(self.token)
        MessageLoop(self.bot, {'chat': self.on_chat_message}).run_as_thread()

    def on_mqtt_connect(self, client, userdata, flags, rc):
        print(f"✅ Bot connected to MQTT broker (rc={rc})")
        client.subscribe(self.topic, qos=1)

    def on_mqtt_message(self, client, userdata, msg):
        try:
            payload = json.loads(msg.payload)

            # Save last observation in the buffer
            self.last_observation.update(payload)

        except Exception as e:
            print(f"Error parsing MQTT message: {e}")

    def on_chat_message(self, msg):
        content_type, chat_type, chat_id = telepot.glance(msg)
        text = msg.get('text', '').strip().lower()

        if text == '/temperature':
            self.send_temperature_status(chat_id)
        elif text == '/shades':
            self.send_shading_status(chat_id)
        elif text == '/help':
            msg = ("Commands:\n"
                "/temperature - Get temperature for all zones\n"
                "/shades - Get current blind positions\n")
            self.bot.sendMessage(chat_id, msg)

    def send_temperature_status(self, chat_id):
        """Fetch latest temperatures for all zones."""
        if not self.last_observation:
            self.bot.sendMessage(chat_id, "No data received yet via MQTT.")
            return

        status_msg = "🌡 **Current Temperatures:**\n"
        for zone in self.zones:
            temp = self.last_observation.get(zone)
            if temp is not None:
                status_msg += f"- {zone.replace('Tair_z', 'Zone ')}: {float(temp):.1f}°C\n"
            else:
                status_msg += f"- {zone.replace('Tair_z', 'Zone ')}: N/A\n"

        print("Sending temperature status:", status_msg)
        self.bot.sendMessage(chat_id, status_msg, parse_mode='Markdown')

    def send_shading_status(self, chat_id):
        """Check if blinds are Open (0) or Closed (7)."""
        if not self.last_observation:
            self.bot.sendMessage(chat_id, "No data received yet via MQTT.")
            return
        
        status_msg = "🪟 **Shading Status:**\n"
        for zone, shade_list in self.shades.items():
            status_msg += f"🏠 **{zone}**\n"
            for shade in shade_list:
                status_val = self.last_observation.get(shade)
                # Displaying 7.0 as Closed and 0.0 as Open per FMU logic
                icon = "🌑" if status_val > 1.0 else "☀️"
                text = "Closed" if status_val > 1.0 else "Open"
                
                # Clean up field name for display (e.g., ShadeStatus_Zone1_Wall2 -> Wall 2)
                label = shade.split('_')[-1]
                status_msg += f"  - {label}: {text} {icon}\n"
            status_msg += "\n"

        print("Sending shading status:", status_msg)
        self.bot.sendMessage(chat_id, status_msg, parse_mode='Markdown')

    def monitor_and_alert(self, chat_id):
        """Check all zones for comfort violations (t_min-t_max°C)."""
        if not self.last_observation:
            self.bot.sendMessage(chat_id, "No data received yet via MQTT.")
            return
        
        # Check for comfort violations
        uncomfort_zones = {}
        for zone in self.zones:
            temp = self.last_observation.get(zone)
            if temp < self.t_min or temp > self.t_max:
                uncomfort_zones.append((zone, float(temp)))
        
        # Send alert if any zone is out of comfort range
        if uncomfort_zones:
            alert_msg = f"⚠️ **Comfort Alert!** The following zones are out of comfort range ({self.t_min}-{self.t_max}°C):\n"
            
            for zone_label, zone_temp in uncomfort_zones.items():
                if zone_temp is not None:
                    alert_msg += f"- {zone_label.replace('Tair_z', 'Zone ')}: {float(zone_temp):.1f}°C\n"
                else:
                    alert_msg += f"- {zone_label.replace('Tair_z', 'Zone ')}: N/A\n"
            
            print("Sending alert:", alert_msg)
            self.bot.sendMessage(chat_id, alert_msg)

In [ ]:
with open("settings.json") as f:
    conf = json.load(f)

bot = NotificationBot(conf)
print("Notification Bot started...")

TEST_CHAT_ID = 7201915868

try:
    while True:
        # Check comfort levels every 60 seconds
        bot.monitor_and_alert(chat_id=TEST_CHAT_ID)
        
        # Wait for 60 seconds before checking again
        time.sleep(60)
except KeyboardInterrupt:
    print("Stopping Bot...")